In [41]:
import pandas as pd
import numpy as np
import nltk
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.utils import to_categorical

In [42]:
# Download NLTK resources (if not already downloaded)
nltk.download('punkt')
nltk.download('wordnet')

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\CPN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\CPN\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [43]:
# Load the dataset
data = pd.read_excel('./final_sentiment.xlsx')

In [44]:
data.head()

,Review,Sentiments
0,I am so angry that i made this post available...,Negative
1,Very bad not recommended,Negative
2,Rooms are nice but for elderly a bit difficul...,Negative
3,My room was dirty and I was afraid to walk ba...,Negative
4,You When I booked with your company on line y...,Negative


In [45]:
data.describe()

,Review,Sentiments
count,139998,139998
unique,104526,2
top,Very bad not recommended,Negative
freq,12348,69999


In [46]:
data.tail()

,Review,Sentiments
139993,It was rather warm and at 10pm noisy with car...,Negative
139994,very small rooms tiny bed supposedly for two ...,Negative
139995,I disliked the package,Negative
139996,Very noisy heater fan and windows letting far...,Negative
139997,The car parking it must to be free for the cu...,Negative


In [47]:
data.columns

Index(['Review', 'Sentiments'], dtype='object')

In [48]:
# Map sentiment labels to numerical values
label_encoder = LabelEncoder()
data['Sentiments'] = label_encoder.fit_transform(data['Sentiments'])


#num_classes = len(label_encoder.classes_)


In [49]:
# Preprocess the data
lemmatizer = WordNetLemmatizer()

In [50]:
def preprocess_text(text):
    # Check if text is a string or bytes-like object
    if isinstance(text, str):
       
        # Tokenize the text
        tokens = word_tokenize(text)

        # Lemmatize the tokens
        lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]

        # Join the tokens back into a string
        processed_text = ' '.join(lemmatized_tokens)

        return processed_text
    else:
        return ''  # Return an empty string for non-string inputs



In [51]:
# Apply preprocessing to the 'Text' column
data['Review'] = data['Review'].apply(preprocess_text)

In [52]:
# Split the data into training and testing sets
train_data, test_data, train_labels, test_labels = train_test_split(data['Review'], data['Sentiments'], test_size=0.2, random_state=42)

In [53]:
from keras.preprocessing.text import Tokenizer
# Tokenize the training data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data)
train_sequences = tokenizer.texts_to_sequences(train_data)
#Tokenization involves converting each word in the text into a unique integer ID. 'this': 1,'is': 2,  'first': 4,

In [54]:
# Tokenize the testing data
test_sequences = tokenizer.texts_to_sequences(test_data)

In [56]:
# Pad the sequences to ensure consistent length
max_sequence_length = 100  # Set the desired maximum sequence length
train_sequences = pad_sequences(train_sequences, maxlen=max_sequence_length)
test_sequences = pad_sequences(test_sequences, maxlen=max_sequence_length)
#The tokenized sequences are padded to ensure a consistent length for each sequence. 

In [57]:
# Convert labels to categorical format
num_classes = len(label_encoder.classes_)
train_labels_categorical = to_categorical(train_labels, num_classes=num_classes)
test_labels_categorical = to_categorical(test_labels, num_classes=num_classes)

#cat:  [1, 0, 0] dog:  [0, 1, 0] cat:  [1, 0, 0] One-hot encoding represents each label as a binary vector of the same length as the number of classes

In [58]:
# Build the LSTM model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_sequence_length))
# we set it to 100, meaning each word will be represented as a 100-dimensional vector.
model.add(LSTM(100))
#The parameter 100 specifies the number of LSTM units or cells in the layer
model.add(Dense(num_classes, activation='softmax'))
#adds a dense layer to the model.
#for  two classes (positive and negative), num_classes would be set to 2.
#The softmax function applies the exponential function and normalizes the values so that they sum up to 1.

In [59]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#loss = - sum(y * log(p) + (1 - y) * log(1 - p))

In [60]:
# Train the model
model.fit(train_sequences, train_labels_categorical, validation_data=(test_sequences, test_labels_categorical), epochs=10)

Epoch 1/10
3500/3500 [==============================] - 462s 131ms/step - loss: 0.1651 - accuracy: 0.9399 - val_loss: 0.1320 - val_accuracy: 0.9520
Epoch 2/10
3500/3500 [==============================] - 637s 182ms/step - loss: 0.1150 - accuracy: 0.9603 - val_loss: 0.1281 - val_accuracy: 0.9546
Epoch 3/10
3500/3500 [==============================] - 643s 184ms/step - loss: 0.0983 - accuracy: 0.9662 - val_loss: 0.1303 - val_accuracy: 0.9548
Epoch 4/10
3500/3500 [==============================] - 477s 136ms/step - loss: 0.0844 - accuracy: 0.9709 - val_loss: 0.1351 - val_accuracy: 0.9546
Epoch 5/10
3500/3500 [==============================] - 419s 120ms/step - loss: 0.0743 - accuracy: 0.9744 - val_loss: 0.1442 - val_accuracy: 0.9551
Epoch 6/10
3500/3500 [==============================] - 436s 125ms/step - loss: 0.0649 - accuracy: 0.9771 - val_loss: 0.1563 - val_accuracy: 0.9512
Epoch 7/10
3500/3500 [==============================] - 1380s 394ms/step - loss: 0.0579 - accuracy: 0.9798 - val

In [61]:
# Measure accuracy
_, train_accuracy = model.evaluate(train_sequences, train_labels_categorical, verbose=0)
_, test_accuracy = model.evaluate(test_sequences, test_labels_categorical, verbose=0)

# Print accuracy
print('Train Accuracy:', train_accuracy)
print('Test Accuracy:', test_accuracy)

Train Accuracy: 0.9862586855888367
Test Accuracy: 0.9488928318023682


In [62]:
# Save the tokenizer and label encoder
tokenizer_path = './tokenizer.pkl'
with open(tokenizer_path, 'wb') as f:
    pickle.dump(tokenizer, f)

In [63]:
label_encoder_path = './label_encoder.pkl'
with open(label_encoder_path, 'wb') as f:
    pickle.dump(label_encoder, f)

In [64]:
# Save the trained model
model_path = 'sentiment_model.h5'
model.save(model_path)

Train Accuracy: 1.0
Test Accuracy: 0.9262697696685791
